In [ ]:
import os
import time
import glob
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

usuario_login = os.getenv("USER_LOGIN")
password_login = os.getenv("PWD_LOGIN")

# definindo caminhos
caminho_raiz = os.getcwd()
caminho_download = os.path.join(caminho_raiz, "notas_fiscais")

# criando a pasta notas_fiscais, caso não exista
os.makedirs(caminho_download, exist_ok=True)

def iniciar_navegador():
  # Configurações para criação e inicialização do navegador
  options = webdriver.ChromeOptions()
  options.add_experimental_option("prefs", {
    "download.default_directory": f"{caminho_download}",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
  })

  return webdriver.Chrome(options=options)

# criando instância do navegador
navegador = iniciar_navegador()

def encontrar_elemento(metodo, seletor, tempo=10):
  # função voltada para auxiliar no momento de encontrar elementos via Selenium
  return WebDriverWait(navegador, tempo).until(EC.presence_of_element_located((metodo, seletor)))

## Preenchendo campos de login/senha e clicando botão login

In [ ]:
def efetuar_login():
    try:
        # Acessando pagina de login
        arquivo_login = os.path.join(caminho_raiz, "login.html")
        navegador.get(rf"file:///{arquivo_login}")

        # preenchendo campo login
        encontrar_elemento(By.XPATH, '/html/body/div/form/input[1]').send_keys(usuario_login)

        # preenchendo campo senha
        encontrar_elemento(By.XPATH, '/html/body/div/form/input[2]').send_keys(password_login)

        # clicando botão login
        WebDriverWait(navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/form/button'))).click()
    except Exception as e:
        print(f'Erro ao efetuar login: {e}')
        navegador.quit()

## Aguardando Download

In [ ]:
def aguardar_download(timeout=30):
    # Aguardar até um arquivo XML ser baixado ou atingir o tempo limite
    tempo_inicio = time.time()
    while time.time() - tempo_inicio < timeout:
        arquivos = glob.glob(os.path.join(caminho_download, "*.xml"))
        # se encontrar algum arquivo XML
        if arquivos:
            return arquivos[0]
        time.sleep(1)
    raise TimeoutError("O Download da nota fiscal não foi concluído a tempo.")

## Preencher dados da NF

In [ ]:
def emitir_nota_fiscal(dados_cliente):
    # Preenche o formulário e emite a nota fiscal com base nos dados do cliente.
    try:
        # criando identificador único com base no timestamp e nome do cliente
        timestamp = datetime.now().strftime("%d%m%Y_%H%M%S")
        id_nota = f"NF_{dados_cliente['Cliente'].replace(' ', '_')}_{timestamp}"

        campos = {
            "nome": dados_cliente["Cliente"],
            "endereco": dados_cliente["Endereço"],
            "bairro": dados_cliente["Bairro"],
            "municipio": dados_cliente["Municipio"],
            "cep": dados_cliente["CEP"],
            "uf": dados_cliente["UF"],
            "cnpj": dados_cliente["CPF/CNPJ"],
            "inscricao": dados_cliente["Inscricao Estadual"],
            "descricao": dados_cliente["Descrição"],
            "quantidade": dados_cliente["Quantidade"],
            "valor_unitario": dados_cliente["Valor Unitario"],
            "total": dados_cliente["Valor Total"],
        }

        # preenchendo os campos do fomulario
        for campo, valor in campos.items():
            encontrar_elemento(By.NAME, campo).send_keys(valor)

        # clicar botão emitir a NF
        WebDriverWait(navegador, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'registerbtn'))).click()

        # aguardar download da NF
        arquivo_nf = aguardar_download()

        data_emissao = datetime.now().strftime("%Y%m%d")
        pasta_data = os.path.join(caminho_download, data_emissao)

        # criar subpasta se nao existir
        os.makedirs(pasta_data, exist_ok=True)

        # renomeando o arquivo NF baixado
        novo_nome_arquivo =  os.path.join(pasta_data, f"{id_nota}.xml")
        if os.path.exists(arquivo_nf):
            os.rename(arquivo_nf, novo_nome_arquivo)
        else:
            print(f'Erro: O arquivo {arquivo_nf} não foi encontrado para renomeação!')

        # recarregando a página para limpar os campos do formulário
        navegador.refresh()
        time.sleep(2)
    except Exception as e:
        print(f'Erro ao emitir nota fiscal para {dados_cliente["Cliente"]}: {e}')
    
    

## Executando código

* efetuando login
* Carregando dados dos clientes
* Emitindo notas fiscais

In [ ]:
def processar_notas():
    try:
        # Efetuando login
        efetuar_login()

        # Carregando dados dos clientes
        tabela_clientes = pd.read_excel("NotasEmitir.xlsx")
        # removendo espaços extras
        tabela_clientes.columns = tabela_clientes.columns.str.strip()
        lista_clientes = tabela_clientes.to_dict(orient="records")

        for cliente in lista_clientes:
            emitir_nota_fiscal(cliente)

    finally:
        # Fechar o navegador ao terminar todo o processo
        navegador.quit()

processar_notas()